# CKP8087 - Estrutura de Dados
<img  src="https://img.shields.io/badge/UFC_CKP8087-VAUX GOMES-000000?style=for-the-badge&logo=" /> <img src="https://img.shields.io/badge/Jupyter-000000?style=for-the-badge&logo=jupyter&logoColor=white" /> <img src="https://img.shields.io/badge/Python-000000?style=for-the-badge&logo=python&logoColor=white" /> 

### Bibliotecas principais

In [48]:
import pandas as pd
import networkx as nx
import random as rnd

### Loading

In [ ]:
df = pd.read_csv('../lista-2/files/dados.csv')
df.rename(columns={'score_sentiment': 'sentiment', 'score_misinformation': 'misinformation'}, inplace=True)

# Flag de viral
counts = df['text'].value_counts()
df['isviral'] = df['text'].isin(counts[counts > 1].index)

# Flaf de misinformation
df['ismisinformation'] = df.misinformation >= 0.7

df.shape

(186316, 7)

In [3]:
df.head(5)

,member,group,text,sentiment,misinformation,isviral,ismisinformation
0,3a8e41b9e1da548ef0acd0a57b398da4,e110071613239754d38878f7e046e95b,Jovem vai a sessão parlamentar na câmara dos v...,0.6371,0.001867,True,False
1,3a8e41b9e1da548ef0acd0a57b398da4,7ee4235534ec624ebd61373b87ad8c20,Jovem vai a sessão parlamentar na câmara dos v...,0.6371,0.001867,True,False
2,3a8e41b9e1da548ef0acd0a57b398da4,ee85f63c945ffa50ba8bb57acf2c1bf9,Jovem vai a sessão parlamentar na câmara dos v...,0.6371,0.001867,True,False
3,ef496106907ece8b169c6219e5470b2c,4450dfcd32cb7582037943ca7c682e2d,*Dilma merece pedido de desculpas por impeachm...,0.4019,NaN,True,False
4,4ea1b3ee637da811b7d2d0df32db21f9,3b75f897e41fca595457f2ad9e101260,*Dilma merece pedido de desculpas por impeachm...,0.4019,NaN,True,False


### Construindo a estrutura de gráfico

In [4]:
geral  = df[['member', 'group', 'isviral', 'ismisinformation']]
viral = geral[geral.isviral]
misin = geral[geral.ismisinformation]

In [32]:
#
def build(df):
    graph = {}
    
    for group in df.group.unique(): 
        members = df[df.group == group].member.value_counts()
        
        for s in members.keys():
            for r in members.keys():
                # Avoiding duplicates
                if s in graph and r in graph and (s in graph[r] or r in graph[s]):
                    continue
    
                graph[s] = graph.get(s, [])
                graph[s].append(r)

    return graph, df.member.unique()

In [33]:
%time graph_geral, memebers_geral = build(geral)
%time graph_viral, memebers_viral = build(viral)
%time graph_misin, memebers_misin = build(misin)

CPU times: user 3.84 s, sys: 24.8 ms, total: 3.87 s
Wall time: 3.87 s
CPU times: user 1.8 s, sys: 12.1 ms, total: 1.81 s
Wall time: 1.83 s
CPU times: user 185 ms, sys: 1.05 ms, total: 186 ms
Wall time: 186 ms


## Parte I - NetworkX

https://networkx.org/

### Funções de montagem

In [7]:
# Função de montagem do grafo
def networkX(graph):
    G = nx.Graph()
    
    for s in graph:
        for r in graph[s]:
            G.add_edge(s, r, color='#0077b6')

    return G

In [93]:
# Montando os grafos
%time gg = networkX(graph_geral)
%time gv = networkX(graph_viral)
%time gm = networkX(graph_misin)

CPU times: user 407 ms, sys: 6.38 ms, total: 414 ms
Wall time: 414 ms
CPU times: user 125 ms, sys: 1.33 ms, total: 126 ms
Wall time: 126 ms
CPU times: user 32.3 ms, sys: 328 µs, total: 32.6 ms
Wall time: 32.6 ms


### Metodologia

- Encontrar o maior component conectado
- Remover 10% de suas arestas aleatóriamente
- Usar os algoritmos e ordenar os _scores_
- Verificar a usabilidade das métricas

#### Giant component

In [94]:
# Giant component
def giant_component(G):
    return G.subgraph(max(nx.connected_components(G), key=len))

# Usando apenas o maior component conectado
%time gg = giant_component(gg)
%time gv = giant_component(gv)
%time gm = giant_component(gm)

print('\nGiant Component Created!')

CPU times: user 38.1 ms, sys: 2.27 ms, total: 40.3 ms
Wall time: 40.1 ms
CPU times: user 10.1 ms, sys: 55 µs, total: 10.2 ms
Wall time: 10.2 ms
CPU times: user 3.14 ms, sys: 224 µs, total: 3.37 ms
Wall time: 3.43 ms

Giant Component Created!


#### Remoção de arestas

In [95]:
def remove_edges(G, perc=0.1):
  n_edges = G.number_of_edges()
  size = int(n_edges * perc)
  ebunch = rnd.choices(list(G.edges), k=size)
  
  H = G.copy()
  H.remove_edges_from(ebunch)
  
  print(f'\nFrom {n_edges:6} to {H.number_of_edges():6}\n--')
  return H, nx.Graph(ebunch)

#
rnd.seed(42)

#
%time gg, gg_rme = remove_edges(gg)
%time gv, gv_rme = remove_edges(gv)
%time gm, gm_rme = remove_edges(gm)

print('\nEdges Removed')


From 479955 to 434334
--
CPU times: user 4.95 s, sys: 27.8 ms, total: 4.98 s
Wall time: 5.01 s

From 142368 to 128787
--
CPU times: user 1.4 s, sys: 7.07 ms, total: 1.4 s
Wall time: 1.42 s

From  34168 to  30904
--
CPU times: user 339 ms, sys: 1.91 ms, total: 341 ms
Wall time: 342 ms

Edges Removed


#### Métricas de predição de ligação

| Métrica | Suporte | |
| -- | -- | -- |
| Preferential attachment | `nativo` | `preferential_attachment()` |
| Common neighbors | `nativo` | `common_neighbors()` |
| Jaccard | `nativo` | `jaccard_coefficient()`|
| Adamic-Adar | `nativo` | `adamic_adar_index()` |
| Graph distance | `desenvolvido` |
| Kat | `nativo` | `katz_centrality()` |